In [5]:
## Different libraries to include - check 
import os
import tensorflow as tf ## Skal installeres på en eller anden måde?
from tensorflow import keras
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import argparse
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.models import Model, model_from_json

from ats.core import attention_sampling, SamplePatches, Expectation
from ats.utils.layers import L2Normalize, SampleSoftmax, ResizeImages, ActivityRegularizer, TotalReshape
from ats.utils.regularizers import multinomial_entropy
from ats.utils.training import Batcher

from DataManager import PathologyData
from ModelManager import AttentionSaver, get_optimizer, get_model, step_decay_schedule, get_configs

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True ## To test on GPU and not CPU? 
session = InteractiveSession(config=config)

ModuleNotFoundError: No module named 'tensorflow'

In [8]:
## Tilføjelse af argparse for at kunne indsætte argumenter i funktioner. Gøres lettere at redigere i optimizers etc
parser = argparse.ArgumentParser()
parser.add_argument("-datapath", help="path to the data set", required=True)
parser.add_argument("-output", help="An output directory", required=True)
parser.add_argument("-set", help="Which dataset are you using? e.g. /* INDSÆT HVILKE IHC VI BRUGER HER! */ ", required=True)

_StoreAction(option_strings=['-set'], dest='set', nargs=None, const=None, default=None, type=None, choices=None, help='Which dataset are you using? e.g. /* INDSÆT HVILKE IHC VI BRUGER HER! */ ', metavar=None)

In [9]:
## String to bool - omdanner en string værdi til boolean / yes or no
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ("yes", "true", "t", "y", "1"):
        return True
    elif v.lower() in ("no", "false", "f", "n", "0"):
        return False
    else:
        raise argparse.ArgumentTypeError("Boolean value expected.")

In [10]:
## Alle parser argumenter Anders bruger - skal kigges igennem for at se hvilke vi synes er vigtige? 
parser.add_argument(
    "-rmbg",
    default=False,
    type=str2bool,
    help="Remove the background from the images before training?"
)
parser.add_argument(
        "-optimizer",
        choices=["sgd", "adam", "rmsprop"],
        default="adam",
        help="Choose the optimizer"
    )
parser.add_argument(
    "-lr",
    help="The Learning Rate for the Optimizer",
    default=0.00001,
    type=float
)
parser.add_argument(
    "-momentum",
    type=float,
    default=0.9,
    help="Choose the momentum for the Optimizer"
)
parser.add_argument(
    "-clipnorm",
    type=float,
    default=1,
    help="Clip the gradient norm to avoid exploding gradients? 1=Y, 0=N"
)
parser.add_argument(
    "-patch_size",
    type=lambda x: tuple(int(xi) for xi in x.split("x")),
    default="100x100",
    help="Choose the size of the patch to extract from the high resolution"
)
parser.add_argument(
    "-n_patches",
    type=int,
    default=50,
    help="How many patches to sample"
)
parser.add_argument(
    "-regularizer_strength",
    type=float,
    default=0.01, #0.01
    help="How strong should the regularization be for the attention"
)

parser.add_argument(
    "-batch_size",
    type=int,
    default=6,
    help="Choose the batch size for SGD"
)
parser.add_argument(
    "-epochs",
    type=int,
    help="How many epochs to train",
    default=200
)
parser.add_argument(
    "-aug",
    type=str2bool,
    default=True,
    help="Do you want to apply data augmentation to the data during training?"
)
parser.add_argument(
    "-cont_train",
    type=str2bool,
    help="Wish to continue training on previous weights?",
    default=False
)
parser.add_argument(
    "-model_dir",
    type=str,
    help="Path to the saved model architectures.",
    required=False
)
parser.add_argument(
    "-att_weights",
    type=str,
    help="Path to the weights of the attention part of the network",
    required=False
)
parser.add_argument(
    "-feat_weights",
    type=str,
    help="Path to the weights for the feature net part of the network.",
    required=False
)
parser.add_argument(
    "-res_depth",
    type=int,
    default=20,
    required=False,
    help="How deep should the ResNet used as feature extraction network be?"
)
parser.add_argument(
    "-res_drop",
    type=str2bool,
    default=False,
    required=False,
    help="Do you want to use dropout layers in your res-net (which serves as your feature extractor)?"
)
parser.add_argument(
    "-res_drop_rate",
    type=float,
    default=0.0,
    required=False,
    help="Set the value of the dropout rate for the dropout layers in your res-net. Only used if \"res_drop\" is True."
)

_StoreAction(option_strings=['-res_drop_rate'], dest='res_drop_rate', nargs=None, const=None, default=0.0, type=<class 'float'>, choices=None, help='Set the value of the dropout rate for the dropout layers in your res-net. Only used if "res_drop" is True.', metavar=None)

In [11]:
## Args er her det man kalder for at hente arg.parse kaldene 

args = vars(parser.parse_args())


network_configs = get_configs(args["set"])

## Pathology data sker her indlæses med vores path. Ændring af kald 
training_set = PathologyData(args["datapath"], set="Train", rm_bg=args["rmbg"],
                             augmentation_mode="standard" if args["aug"] else "",
                             classes_dict=network_configs["classes"])
training_batched = Batcher(training_set, batch_size=args["batch_size"])

## Samme her PathologyData
validation_set = PathologyData(args["datapath"], set="Val", rm_bg=args["rmbg"],
                               augmentation_mode="",
                               classes_dict=network_configs["classes"])
validation_batched = Batcher(validation_set, batch_size=args["batch_size"])

## Get_model til crop_reshape
if not args["cont_train"]:
    model, att_model = get_model(
        outputs=len(training_set.CLASSES),
        width=network_configs["crop_reshape"][0],
        height=network_configs["crop_reshape"][1],
        scale=0.05,
        res_depth=args["res_depth"],
        res_drop=args["res_drop"],
        res_drop_rate=args["res_drop_rate"],
        n_patches=args["n_patches"],
        patch_size=args["patch_size"],
        reg=args["regularizer_strength"]
    )

usage: ipykernel_launcher.py [-h] -datapath DATAPATH -output OUTPUT -set SET [-rmbg RMBG]
                             [-optimizer {sgd,adam,rmsprop}] [-lr LR] [-momentum MOMENTUM] [-clipnorm CLIPNORM]
                             [-patch_size PATCH_SIZE] [-n_patches N_PATCHES]
                             [-regularizer_strength REGULARIZER_STRENGTH] [-batch_size BATCH_SIZE] [-epochs EPOCHS]
                             [-aug AUG] [-cont_train CONT_TRAIN] [-model_dir MODEL_DIR] [-att_weights ATT_WEIGHTS]
                             [-feat_weights FEAT_WEIGHTS] [-res_depth RES_DEPTH] [-res_drop RES_DROP]
                             [-res_drop_rate RES_DROP_RATE]
ipykernel_launcher.py: error: the following arguments are required: -datapath, -output, -set


SystemExit: 2

C:\Users\andre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
## Compile af modellen
model.compile(
    loss="categorical_crossentropy",
    optimizer=get_optimizer(opt_choice=args["optimizer"], learning_rate=0.0, momentum=args["momentum"], clipnorm=args["clipnorm"]),
    metrics=["accuracy"]
)


NameError: name 'model' is not defined

In [ ]:
## Send model til json_fil 